In [1]:
import imdb as imdb#to access imdb API
import pandas as pd #for data array handling
from bs4 import BeautifulSoup as soup
# for website parsing and scraping (rotten tomatoes)
from urllib.request import urlopen as uReq
import re#for reqular expressions
#from pandas.lib import Timestamp
from ggplot import * #for plotting


import matplotlib.pyplot#for plotting
from matplotlib.offsetbox import (OffsetImage, AnnotationBbox)




C:\Users\user\AppData\Roaming\Python\Python36\site-packages\ggplot\utils.py:81: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  pd.tslib.Timestamp,


In [2]:
#get the imdb rate in each episode

#pandas의 datframe을 사용했음
### pandas의 dataframe은 테이블 형식의 data를 다룰 때 사용. data를 읽어들여 pandas상의 table에 저장

filename= "episodes.csv"
filename2="seasons.csv"
f=open(filename, "w")
f2=open(filename2, "w")




In [3]:
from pandas import Series,DataFrame


In [4]:
#scraping IMDb to get the data
j=1
a=0
b=0
k=0
df_e=pd.DataFrame()
df_s=pd.DataFrame({'Season': [1,2,3,4,5]})

for i in range(1,6):
    SeasonAVG=0
    season_avg=0       
    idx=0
    episode_num=0
    df_e = df_e.append({'Season': i},ignore_index=True)
    imdb_url = 'https://www.imdb.com/title/tt2575988/episodes?season=%d' %i
    iClient = uReq(imdb_url) #opening the connection, downloading the webpage
    ipage_html = iClient.read()
    iClient.close()
    
    #do html parsing
    ipage_soup = soup(ipage_html,"html.parser")
    episodenames = ipage_soup.findAll("a",{"itemprop":"name"})
    rates = ipage_soup.findAll("span",{"class":"ipl-rating-star__rating"})
    for episodename in episodenames:
            k+=1
            if k<48:
                ep_name = episodename["title"]
                df_e = df_e.append( {'Episode_Name':ep_name},ignore_index=True)
                df_e["Episode_Name"][a]=ep_name
                a+=1
                for rate in rates:
                    if idx==0 or idx%23==0:
                        imdb_rate = rate.text
                        season_avg+=float(imdb_rate)
                        df_e = df_e.append( {"IMDb_rate":imdb_rate},ignore_index=True)
                        
                        df_e["IMDb_rate"][b]=imdb_rate
                        b+=1
                        episode_num+=1
                    idx+=1
    
    SeasonAVG=float(season_avg/episode_num)
    df_s = df_s.append( {"IMDb_season_avg":SeasonAVG},ignore_index=True)
    df_s["IMDb_season_avg"][i-1]=SeasonAVG
                


C:\Users\user\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
df_e.head(30) 
#df_e.to_csv("episodes.csv",index= False,mode='w')

,Season,Episode_Name,IMDb_rate
0,1.0,Minimum Viable Product,7.8
1,NaN,The Cap Table,7.9
2,NaN,Articles of Incorporation,8.2
3,NaN,Fiduciary Duties,7.8
4,NaN,Signaling Risk,8.4
5,NaN,Third Party Insourcing,8.5
6,NaN,Proof of Concept,8.6
7,NaN,Optimal Tip-To-Tip Efficiency,9.4
8,NaN,Sand Hill Shuffle,7.8
9,NaN,Runaway Devaluation,7.9


In [6]:
#scraping ROTTEN TOMATOES to get the data
my_url = 'https://www.rottentomatoes.com/tv/silicon_valley'
uClient = uReq(my_url) #opening the connection, downloading the webpage
page_html = uClient.read()
uClient.close()
#doing html parsing
page_soup = soup(page_html,"html.parser")


In [7]:
tomatometer = page_soup.findAll("span",{"class":"meter-value"})
#tomatometer
#grab each tomatometer

In [8]:
#Tomatometers = tomatometer[0] #prototyping the loop
i=6
for Tomatometers in tomatometer:
    TM = tomatometer[i].text
    i=i-1
    if i<=0:break
    df_s = df_s.append( {"Tomatometers":TM},ignore_index=True)
    df_s["Tomatometers"][i-1] = TM
#df_s.head(30)
#df_s.to_csv("seasons.csv",index= False, mode='w')


C:\Users\user\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
#put the vieweres statistic in csv file, and then concatenate it with episodes.csv and seasons.csv
#df_v=pd.DataFrame()
df_v = pd.read_csv("viewers.csv",names=["viewers"])
df_va = pd.read_csv("viewers_avg.csv",names=["viewers_season_avg"])
df_ev = pd.concat([ df_e,df_v],axis=1)
df_sva = pd.concat([df_s,df_va],axis =1)




In [11]:
df_e.to_csv("episodes.csv",mode='w')

df_s.to_csv("seasons.csv", mode='w')

df_ev.to_csv("episodes_all.csv",mode='w')





In [14]:
df_sva.to_csv("seasons_all.csv", mode='w')

In [12]:
f2.close()
f.close()